In [59]:
import plotly.graph_objects as go
import dash
import plotly.express as px

import dash_html_components as html

import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.offline as pyo
import numpy as np
import pandas as pd

In [ ]:
app  = dash.Dash()
country_codes = ['BR', 'CA', 'DE', 'FR', 'GB', 'IN', 'JP', 'KR', 'MX', 'RU', 'US']
options_list = []
for i in country_codes:
    options_list.append({'label': i, 'value': i})

df_test  = pd.read_csv("Dataset/Titledata/BR_title_totals.csv")

total_titles = np.array(df_test.total_number_of_titles)[0]
did_use_par_or_bracks = (np.array(df_test.number_of_titles_with_parenthesis_or_squarebracket_usage)[0]/total_titles)*100
did_use_caps = (np.array(df_test.number_of_titles_with_caps_usage)[0]/total_titles)*100
did_use_emojis = (np.array(df_test.number_of_titles_with_emoji_usage)[0]/total_titles)*100

did_not_use_par_or_bracks = 100 - did_use_par_or_bracks
did_not_use_caps = 100 - did_use_caps
did_not_use_emojis = 100 - did_use_emojis

x = ['Did use () or []', 'Did use CAPS', 'Did use emojis']

fig = go.Figure()
fig.add_trace(go.Bar(name="Yes", x=x, y=[did_use_par_or_bracks, did_use_caps, did_use_emojis]))
fig.add_trace(go.Bar(name="No", x=x, y=[did_not_use_par_or_bracks, did_not_use_caps, did_not_use_emojis]))

fig.update_layout(title_text='Titles')
fig.update_layout(barmode='stack')
fig.update_layout(yaxis_range=(0, 100))



app.layout = html.Div(children=[
    html.Div(
        dcc.Dropdown(id="title_drop_down", options=options_list, value='c'),
             style={"text-align": "center", "display": "inline-block", "width":"100%"}),
    html.Div(dcc.Graph(id='example-graph',figure=fig),
             style={"text-align": "center", "display": "inline-block", "width":"100%"})
], style={"width": "800px", "border": "1px solid black"})


@app.callback(
    Output("plot_area","children"),
    Input("title_drop_down", "value"))
def update_output_div(input_value):
    return 'Output: {}'.format(input_value)

if __name__ == '__main__' :
    app.run_server()


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

INFO:__main__:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
INFO:werkzeug: * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Nov/2021 13:33:01] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2021 13:33:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2021 13:33:01] "GET /_dash-layout HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2021 13:33:01] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2021 13:33:01] "GET /_dash-dependencies HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2021 13:33:01] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2021 13:33:01] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2021 13:33:01] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2021 13:33:01] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2021 13:33:01] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 